In [2]:
import os
import math
import talib
import random
import pickle
import json
import pandas as pd
import numpy as np
import jqdatasdk as jq
import baostock as bs
import time

from scipy import signal
from scipy import stats
from datetime import datetime, timedelta
from tqdm import tqdm

import matplotlib.pyplot as plt
from IPython.core.display import HTML, clear_output

%matplotlib inline
%config InlineBackend.figure_format='svg'
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',    5000)

In [3]:
from featureutils import FeatureTool
from plotutils import PlotTool
from datagent import BaosAgent
from datagent import DataUtil
from datagent import JQDataUtil
from sfstrategy import SFStrategy

In [4]:
def get_freq_file_sto_ids(freq):
    file_sto_ids = []
    path_src = './data/' + str(freq) + '/'
    files = os.listdir(path_src)
    all_file_sto_id = []
    for file in files:
        if file.endswith('pkl'):
            file_sto_ids.append(file[:6])

    return file_sto_ids

In [363]:
jq.auth('15808061188', 'allan2jq')
clear_output()
print(jq.get_query_count())
jq.logout()

{'total': 10000000, 'spare': 9805303}
已退出


In [ ]:
def obs(sto_id, delta, save_flag, pic_path):
    daily_count = int(240 / delta)
    minimum = daily_count * 5 * 24

    df = BaosAgent.read_freq_df(delta, sto_id)
#     ts_jq = JQDataUtil.get_end_ts(delta)
#     df = DataUtil.add_rt_data(sto_id, df, delta, ts_jq)
    
    minp = min(df.iloc[-minimum:].close)
    curp = df.iloc[-1].close
    cr = (curp - minp) / minp
    
    df = FeatureTool.add_ta_features(df, daily_count)
    df = FeatureTool.add_close_features(df, daily_count)
    df = FeatureTool.add_volume_features(df, daily_count)
    df = df.dropna()
    df = df.reset_index(drop=True)
    print('df size: ' + str(len(df)))
    if len(df) >= minimum and 5 < df.close.iloc[-1] < 50:
        df = df.iloc[-minimum:]
        df = df.reset_index(drop=True)
        
        cond = SFStrategy.get_cond_k12345above80_last40(df, daily_count)
        tgt_idxes = df[cond].index
        if len(tgt_idxes) > 0:
            print('{:>6.2%} {} {} {} {} {:>6.2f}'.format(cr, sto_id, len(df), df.time.iloc[tgt_idxes[-1]], df.time.iloc[-1], df.close.iloc[-1]))
        else:
            print('{:>6.2%} {} {} {} {:>6.2f}'.format(cr, sto_id, len(df), df.time.iloc[-1], df.close.iloc[-1]))
        PlotTool.display_obs(df, daily_count, tgt_idxes, save_flag, pic_path)
    return df

In [321]:
sto_id = random.sample(sto_ids, 1)[0]
df = BaosAgent.read_freq_df(15, sto_id)
df.time.iloc[-1]

'202002071500'

In [318]:
len(sto_ids)

3448

In [5]:
delta = 15
sto_ids = get_freq_file_sto_ids(delta)
df = BaosAgent.read_freq_df(delta, random.sample(sto_ids, 1)[0])
df.iloc[-1].time

'202002211500'

In [ ]:
delta = 15
matches_df = pd.read_csv('matches.csv', dtype={'sto_id':str})
sto_ids_backup = matches_df.sto_id.tolist()
sto_ids_backup.sort()
print(len(sto_ids_backup))
for sto_id in sto_ids_backup:
    obs_rt(sto_id, delta, True)
    clear_output()

In [ ]:
delta = 15
sto_ids = get_freq_file_sto_ids(delta)

In [ ]:
for sto_id in random.sample(sto_ids, 200):
    df = obs(sto_id, delta, True, './pic/')
    clear_output()

In [ ]:
delta = 15
sto_ids = get_freq_file_sto_ids(delta)
sto_id = random.sample(sto_ids, 1)[0]
# sto_id = '603527'
df = obs(sto_id, delta, False, './pic/')

In [ ]:
delta = 15
sto_ids = get_freq_file_sto_ids(delta)
sto_id = random.sample(sto_ids, 1)[0]

    daily_count = int(240 / delta)
    minimum = daily_count * 5 * 28
    ts_jq = JQDataUtil.get_end_ts(delta)
    df = BaosAgent.read_freq_df(delta, sto_id)
    df = DataUtil.add_rt_data(sto_id, df, delta, ts_jq)
    
    minp = min(df.iloc[-minimum:].close)
    curp = df.iloc[-1].close
    cr = (curp - minp) / minp
    
    df = FeatureTool.add_ta_features(df, daily_count)
    df = FeatureTool.add_close_features(df, daily_count)
    df = FeatureTool.add_volume_features(df, daily_count)
    df = df.dropna()
    df = df.reset_index(drop=True)

In [ ]:
jq.auth('15808061188', 'allan2jq')
clear_output()
jq.get_query_count()

In [ ]:
delta = 15
daily_count = int(240/delta)

sto_ids = get_freq_file_sto_ids(delta)
sto_id = random.sample(sto_ids, 1)[0]


df = BaosAgent.read_freq_df(delta, sto_id)
df = DataUtil.add_rt_data(sto_id, df, delta, JQDataUtil.get_end_ts(delta))
df = FeatureTool.add_bband_ft(df, daily_count, 70)
df = FeatureTool.add_kd_ft(df, daily_count, 70)
df = df.dropna()
df = df.reset_index(drop=True)

tgt_idxes = []
# cond = df.K_5.shift(1) < df.D_5.shift(1)
# cond = (df.K_5 > df.D_5) & cond 
# cond = (df.K_5 < 20) & (df.D_5 < 20) & cond

# cond = (df.BAND_upper > df.BAND_upper.shift(1))
# cond = (df.BAND_middle > df.BAND_middle.shift(1)) & cond
# cond = (df.BAND_lower > df.BAND_lower.shift(1)) & cond

cond = df.close > 0
for col in df.columns:
    if col.startswith('BAND_'):
        cond = (df[col] > df.close) & cond
        
tgt_idxes = df[cond].index
print(tgt_idxes)

fig = plt.figure(figsize=(PlotTool.fig_width_factor * len(df) / (daily_count * 30), PlotTool.fig_height * 3))
fig.patch.set_facecolor('#2A2A2A')
xaxis_idxes = df.index.values
x_axis_slice = slice(0, -1, daily_count * 5)

ax = plt.subplot(1, 1, 1)
PlotTool.custom_ax(ax)
ax.plot(xaxis_idxes, df.close, lw=.5, color='orange')
for col in df.columns:
    if col.startswith('BAND_upper'):
        ax.plot(xaxis_idxes, df[col],  lw=.1, color='orangered')
    if col.startswith('BAND_middle'):
        ax.plot(xaxis_idxes, df[col],  lw=.1, color='orange')
    if col.startswith('BAND_lower'):
        ax.plot(xaxis_idxes, df[col],  lw=.1, color='lime')

# ax.plot(xaxis_idxes, df['K_1'],  lw=.5, color='cyan')
# ax.plot(xaxis_idxes, df['D_1'],  lw=.2, color='cyan')
# ax.hlines(20, np.min(xaxis_idxes), np.max(xaxis_idxes), lw=.3, color='w')
# ax.hlines(80, np.min(xaxis_idxes), np.max(xaxis_idxes), lw=.3, color='w')
for obs_idx in tgt_idxes:
    ax.vlines(xaxis_idxes[obs_idx], np.min(df.close), np.max(df.close), lw=.2, color='w')
plt.sca(ax)
plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w')
plt.yticks(fontsize=8, color='w')
# ax2 = ax.twinx()
# PlotTool.custom_ax(ax2)
# ax2.plot(xaxis_idxes, df.close, lw=.5, color='orange')
# for obs_idx in tgt_idxes:
#     ax2.vlines(xaxis_idxes[obs_idx], np.min(df.close), np.max(df.close), lw=.2, color='w')
# plt.sca(ax2)
# plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w')
# plt.yticks(fontsize=8, color='w')

plt.show()
plt.close('all')

# observe

In [361]:
for sto_id in sto_ids:
    df = BaosAgent.read_freq_df(delta, sto_id)
    if '202002111500' != df.time.iloc[-1]:
        print(sto_id, df.time.iloc[-1])

300104 201904251500
300216 201904251500
300028 201808071500
300353 202001201500
000029 201609131500


In [ ]:
def generate_target_pic(sto_id, delta, pic_path, ts_jq, pic_fn):
    
    daily_count = int(240/delta)
    df = BaosAgent.read_freq_df(delta, sto_id)
    df = DataUtil.add_rt_data(sto_id, df, delta, ts_jq)
    
    df = FeatureTool.add_ta_features(df, daily_count)
    df = FeatureTool.add_close_features(df, daily_count)
    df = FeatureTool.add_volume_features(df, daily_count)

    df = df.dropna()
    df = df.iloc[-daily_count * 5 * 24:]
    df = df.reset_index(drop=True)
    print(len(df))
    try:
        cond = SFStrategy.get_cond_k12345above80_last40(df, daily_count)
        match_idxes = list(df[cond].index)
        match_size = len(match_idxes)
        print(match_idxes)

        minp = min(df.close)
        curp = df.iloc[-1].close
        cr = (curp - minp) / minp
        print('{} {} {:>6.2%} {:>5.2f} {:>5.2f}'.format(sto_id, df.time.iloc[match_idxes[-1]], cr, minp, curp))
        print()
        
        PlotTool.display_obs(df, daily_count, match_idxes, pic_fn, True, pic_path)
        clear_output()
    except Exception as e:
        print(e)
        pass

jq.auth('15808061188', 'allan2jq')

delta = 15
target_path = './target/'
target_fns = os.listdir(target_path)
target_fns.sort()
for target_fn in target_fns:
    
    if not target_fn.startswith('matches'):
        continue
    if os.path.isdir(target_path + target_fn):
        continue
    
    pic_path = target_path + target_fn[:-4] + '/'
    ts_jq = datetime.strptime(target_fn[8:-4], '%Y-%m-%d-%H%M')
    ts_jq = datetime.strftime(ts_jq, '%Y-%m-%d %H:%M:%S')
    if not os.path.exists(pic_path):
        os.mkdir(pic_path)
        target_df = pd.read_csv(target_path + target_fn, dtype={'sto_id':str})
        for idx in target_df.index:
            record = target_df.iloc[idx]
            sto_id = record.sto_id
            lastest_match_ts = record.lastest_match_ts
            last_20weeks_cr = record.last_20weeks_cr
            last_week_cr = record.last_week_cr
            pic_fn = '{}_{}_{:>6.2%}_{:>6.2%}.png'.format(sto_id, lastest_match_ts, last_20weeks_cr, last_week_cr)
            generate_target_pic(sto_id, delta, pic_path, ts_jq, pic_fn)
jq.logout()

In [ ]:
delta = 15
daily_count = int(240 / delta)
# sto_ids = get_freq_file_sto_ids(delta)
sto_id = random.sample(sto_ids_backup, 1)[0]
# sto_id = '002436'

In [ ]:
# idx += 1
# sto_id = sto_ids_backup[idx]

delta = 15
daily_count = int(240 / delta)
minimum_df_size = daily_count * 5 * 30
for sto_id in sto_ids_backup:
    df = BaosAgent.read_freq_df(delta, sto_id)
#     df = DataUtil.add_rt_data(sto_id, df, delta)
    
    df = FeatureTool.add_ta_features(df, daily_count)
    df = FeatureTool.add_close_features(df, daily_count)
    df = FeatureTool.add_volume_features(df, daily_count)

    df = df.dropna()
    df = df.iloc[-daily_count * 5 * 30:]
    df = df.reset_index(drop=True)
    print(len(df), minimum_df_size == len(df))
    
    try:
        cond = SFStrategy.get_cond_k12345above80_last40(df, daily_count)
        # cond = self.get_cond_ksumavg_cross60(df)
        match_idxes = list(df[cond].index)
        match_size = len(match_idxes)
        print(match_idxes)

        minp = min(df.close)
        curp = df.iloc[-1].close
        cr = (curp - minp) / minp
        print('{} {} {:>6.2%} {:>5.2f} {:>5.2f}'.format(sto_id, df.time.iloc[match_idxes[-1]], cr, minp, curp))
        print()
        
        PlotTool.display_obs(df, daily_count, match_idxes, True)
        clear_output()
    except Exception as e:
        print(e)
        pass

In [ ]:
# jq.auth('15808061188', 'allan2jq')
sfs = SFStrategy(15)
num_processes = 6
sfs.find_matches(num_processes)

In [ ]:
sfs = SFStrategy(15)
num_processes = 6
sfs.find_matches(num_processes)

In [ ]:
delta = 15
daily_count = int(240 / delta)
sto_ids = get_freq_file_sto_ids(delta)
sto_id = random.sample(sto_ids, 1)[0]

df = BaosAgent.read_freq_df(delta, sto_id)
df = FeatureTool.add_ta_features(df, daily_count)

In [ ]:
np.nanmax(close_slopes), np.nanmin(close_slopes)

In [ ]:
delta = 15
daily_count = int(240/delta)
sto_ids = get_freq_file_sto_ids(delta)
sto_id = random.sample(sto_ids, 1)[0]

df = BaosAgent.read_freq_df(delta, sto_id)


df = FeatureTool.add_ta_features(df, daily_count)
df = FeatureTool.add_close_features(df, daily_count)
df = FeatureTool.add_volume_features(df, daily_count)

df = df.iloc[-daily_count * 5 * 50:]

df = df.dropna()
df = df.reset_index(drop=True)

cond = ((df.K_1 > 70) | (df.K_2 > 70) | (df.K_3 > 70) | (df.K_4 > 70) | (df.K_5 > 70)).rolling(daily_count * 5).apply(lambda x: x.all(), raw=True) == 1
# cond = ((df.MFISUM_AVG > 50).rolling(daily_count * 2).apply(lambda x: x.all(), raw=True) == 1) & cond
# cond = df.MFISUM_AVG > 50
# cond = (df.MFISUM_AVG > df.MFISUM_AVG.shift(1)) & cond
# cond = (df.KSUM_AVG > df.KSUM_AVG.shift(1)) & cond
# cond = (df.MFISUM_AVG.diff(1) < 0) & (df.MFISUM_AVG.diff(-1) < 0) & cond
# cond = (df.K_1.diff(1) < 0) & (df.K_1.diff(-1) < 0) & cond

# cond = (df.KSUM_AVG > df.KSUM_AVG.shift(1))
# cond = (df.slope_MFISMU_AVG > 0) & (df.slope_close < 0) & cond

# cond = df.KSUM_AVG < 20

match_idxes = df[cond].index
print(len(df), len(df[cond]))

PlotTool.display_obs(df, daily_count, match_idxes, '', False, '')

In [ ]:
k